# Initial step

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/home/anais/code/anaisdangeot/mood_detector/raw_data/labeled_lyrics_cleaned_processed.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,artist,seq,song,label,cleaned_lyrics
0,0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626,aint ever trap bando oh lord dont get wrong kn...
1,1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630,drink go smoke go feel get let go care get los...
2,2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240,dont live planet earth find love venus thats w...
3,3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536,trippin grigio mobbin light low trippin grigio...
4,4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371,see midnight panther gallant brave find find a...


In [3]:
data['cleaned_lyrics'].isna().sum()
data['cleaned_lyrics'] = data['cleaned_lyrics'].dropna()

In [4]:
data_sub = data.sample(10000)

In [5]:
def cat_valence(row):
    if row >= 0.5:
        return 1
    elif row <0.5:
        return 0
    else:
        return None
data_sub['mood'] = data_sub['label'].apply(lambda x:cat_valence(x))
data_sub['mood'].value_counts()

0    5190
1    4810
Name: mood, dtype: int64

In [6]:
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


# Feature/Target
X = data_sub['cleaned_lyrics'].apply(lambda x: np.str_(x))
y = data_sub["mood"]

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

# Baseline using logistic regression

import numpy as np

from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score

# Pipeline vectorizer + Naive Bayes
pipeline = make_pipeline(
    TfidfVectorizer(), 
    LogisticRegression()
)

# Cross-validation
cv_results = cross_validate(pipeline, X_train, y_train, cv = 5, scoring = ["accuracy"])
average_accuracy = cv_results["test_accuracy"].mean()
baseline_accuracy = np.round(average_accuracy,2)
baseline_accuracy

from sklearn.model_selection import GridSearchCV

# Define the grid of parameters
parameters = {
    'tfidfvectorizer__ngram_range': ((2,2), (1,2)),
    'tfidfvectorizer__max_df': [0,25, 0.3, 0.35],
    'tfidfvectorizer__max_features': [50],
}

# Perform Grid Search
grid_search = GridSearchCV(
    pipeline,
    parameters,
    scoring = "accuracy",
    cv = 5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

# Best score
print(f"Best Score = {grid_search.best_score_}")

# Best params
print(f"Best params = {grid_search.best_params_}")

After different iterations, best params: max_df= 0.35, max_features=50, n_gram=(1,2)

## Preparation for analysis

In [7]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.35,max_features=50)

vectorized_documents = vectorizer.fit_transform(X_train)

In [8]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

### Let's create some mock data
def get_mock_up_data():

    ### Let's tokenize the vocabulary 
    tk = Tokenizer()
    tk.fit_on_texts(X_train)
    vocab_size = len(tk.word_index)
    print(f'There are {vocab_size} different words in your corpus')
    X_token = tk.texts_to_sequences(X_train)

    ### Pad the inputs
    X_pad = pad_sequences(X_token, dtype='float32', padding='post')
    
    return X_pad, vocab_size

X_pad, vocab_size = get_mock_up_data()
print("X_pad.shape", X_pad.shape)
X_pad

2023-06-15 07:44:39.877275: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-15 07:44:40.258316: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-15 07:44:40.261387: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-15 07:44:43.269326: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


There are 26275 different words in your corpus
X_pad.shape (7000, 637)


array([[2.680e+02, 2.380e+02, 4.200e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [7.620e+03, 8.977e+03, 8.978e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [3.500e+01, 1.000e+00, 5.400e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [3.910e+02, 3.910e+02, 5.400e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [5.500e+01, 5.200e+01, 5.000e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [4.300e+01, 7.300e+01, 4.200e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]], dtype=float32)

# DL model using RNN (LSTM)

In [9]:
### Let's build the neural network now
from tensorflow.keras import layers, Sequential

# Size of your embedding space = size of the vector representing each word
embedding_size = 100

model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size+1, #+1 for the 0 padding
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer :)
))

model.add(layers.LSTM(20))
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         2627600   
                                                                 
 lstm (LSTM)                 (None, 20)                9680      
                                                                 
 dense (Dense)               (None, 1)                 21        
                                                                 
Total params: 2,637,301
Trainable params: 2,637,301
Non-trainable params: 0
_________________________________________________________________


In [10]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='loss', patience=10)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


history = model.fit(X_pad, y_train, epochs=100, validation_split=0.2, batch_size=32, verbose=1, callbacks=[es])

Epoch 1/100
175/175 [==============================] - 128s 700ms/step - loss: 0.6796 - accuracy: 0.5720 - val_loss: 0.6683 - val_accuracy: 0.6100
Epoch 2/100
175/175 [==============================] - 109s 624ms/step - loss: 0.6154 - accuracy: 0.6721 - val_loss: 0.6520 - val_accuracy: 0.6121
Epoch 3/100
175/175 [==============================] - 104s 595ms/step - loss: 0.5574 - accuracy: 0.7300 - val_loss: 0.6962 - val_accuracy: 0.6164
Epoch 4/100
175/175 [==============================] - 104s 595ms/step - loss: 0.5022 - accuracy: 0.7713 - val_loss: 0.6980 - val_accuracy: 0.6050
Epoch 5/100
175/175 [==============================] - 103s 586ms/step - loss: 0.4577 - accuracy: 0.8020 - val_loss: 0.7524 - val_accuracy: 0.5929
Epoch 6/100
175/175 [==============================] - 101s 580ms/step - loss: 0.4093 - accuracy: 0.8288 - val_loss: 0.7999 - val_accuracy: 0.5900
Epoch 7/100
175/175 [==============================] - 96s 549ms/step - loss: 0.3729 - accuracy: 0.8477 - val_loss: 0.

In [ ]:
model.evaluate(X_test,y_test)

# DL using CNN

embedding_size = 100
model_cnn = Sequential([
    layers.Embedding(input_dim=7000, input_dim=vocab_size+1, output_dim=embedding_size, mask_zero=True),
    layers.Conv1D(20, kernel_size=3),
    layers.Flatten(),
    layers.Dense(1, activation="sigmoid"),
])

es = EarlyStopping(monitor='loss', patience=1)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


history = model.fit(X_pad, y_train, epochs=100, validation_split=0.2, batch_size=64, verbose=1, callbacks=[es])

# ML using SVM classifier

from sklearn.svm import SVC
svc = SVC(kernel='linear', C=10)

# Pipeline vectorizer + Naive Bayes
pipeline_SVM = make_pipeline(
    TfidfVectorizer(), 
    SVC()
)

# Cross-validation
cv_results = cross_validate(pipeline_SVM, X_train, y_train, cv = 5, scoring = ["accuracy"])
SVM_average_accuracy = cv_results["test_accuracy"].mean()
SVM_accuracy = np.round(SVM_average_accuracy,2)
SVM_accuracy

# Define the grid of parameters
parameters = {
    'SVC__kernel': ('linear', 'poly', 'rbf', 'sigmoid'),
    'SVC__C': (0.2, 0.5, 0.7)
}

# Perform Grid Search
grid_search = GridSearchCV(
    pipeline,
    parameters,
    scoring = "accuracy",
    cv = 5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

# Best score
print(f"Best Score = {grid_search.best_score_}")

# Best params
print(f"Best params = {grid_search.best_params_}")